In [1]:
import numpy as np
import json

In [2]:
from tensorflow.keras.models import load_model


In [3]:
import nltk
from nltk.stem import WordNetLemmatizer

In [4]:
import pickle
import random

In [5]:
lemmatizer = WordNetLemmatizer()
a = json.loads(open('jsonData.json').read())

In [6]:
words = pickle.load(open('words.pkl', 'rb'))
classes = pickle.load(open('classes.pkl', 'rb'))

In [7]:
model = load_model('chatbotmodel.h5')

In [8]:
type(model)

tensorflow.python.keras.engine.sequential.Sequential

In [9]:
def clean_up_sentences(sentence):
    sentence_words = nltk.tokenize.word_tokenize(sentence)
    sentence_words = [lemmatizer.lemmatize(word) for word in sentence_words]
    return sentence_words

In [10]:
def bag_of_word(sentence):
    sentence_words = clean_up_sentences(sentence)
    bag = [0] * len(words)
    for w in sentence_words:
        for i,word in enumerate(words):
            if word == w:
                bag[i] = 1
    return np.array(bag)

In [11]:
def predict_class(sentence):
    bow = bag_of_word(sentence)
    res = model.predict(np.array([bow]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i, r] for i,r in enumerate(res) if r > ERROR_THRESHOLD]
    
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({'intent': classes[r[0]], 'probability':str(r[1])})
    return return_list

In [12]:
def get_response(intents_list, intents_json):
    tag = intents_list[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if i['tag'] == tag:
            result = random.choice(i['responses'])
            break
    return result

In [14]:
while True:
    message = input("")
    ints = predict_class(message)
    res = get_response(ints, a)
    print(res)
    if message == 'bye':
        break

hey
hello
what are charges for savings account
Average balance up to TK.5000/- : No Charges, Average Balance between TK.5,000/- and TK.25,000/-: Tk.100/- half yearly Average Balance above TK.25,000/ -: TK.300/ -half yearly.You can visit our website for more information
i lost my locker key
To find a new locker key you must file an application to our nearby branch .Plese visit our nearby branch. You can visit our website for more information
bye
hey
